In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

RAW_DIR = Path("../data/raw")
OUT_DIR = Path("../data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [2]:
csv = next(RAW_DIR.glob("StormEvents_details-*.csv"), None)
assert csv is not None, "Put your StormEvents_details-*.csv into data/raw/ first."
df = pd.read_csv(csv, low_memory=False)
print(csv, df.shape)


..\data\raw\StormEvents_details-ftp_v1.0_d2025_c20250818.csv (33904, 51)


In [3]:
df.columns = df.columns.str.strip().str.lower()  # lower-case, no stray spaces
print(df.shape)
df.head()

(33904, 51)


,begin_yearmonth,begin_day,begin_time,end_yearmonth,end_day,end_time,episode_id,event_id,state,state_fips,...,end_range,end_azimuth,end_location,begin_lat,begin_lon,end_lat,end_lon,episode_narrative,event_narrative,data_source
0,202503,31,1104,202503,31,1106,201366,1252415,GEORGIA,13,...,2.0,W,TYUS,33.4757,-85.238,33.4757,-85.238,A cold-front initiated a line of thunderstorms...,Tree down at the intersection of highway 5 and...,CSV
1,202503,30,1552,202503,30,1555,200337,1241136,MICHIGAN,26,...,1.0,NNE,EDWARDSBURG,41.7900,-86.100,41.8200,-86.070,A cold front pushed into the area during the a...,A brief EF-1 tornado was confirmed in Edwardsb...,CSV
2,202501,5,1800,202501,6,2227,197733,1222851,VIRGINIA,51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,An area of low pressure tracked across souther...,NaN,CSV
3,202501,3,1300,202501,3,1900,197761,1223112,MARYLAND,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,An area of low pressure moved off into New Eng...,NaN,CSV
4,202501,3,1300,202501,3,1900,197761,1223113,MARYLAND,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,An area of low pressure moved off into New Eng...,NaN,CSV


In [4]:
# Drop exact duplicate rows
before = len(df)
df = df.drop_duplicates()
print(f"Duplicates removed: {before - len(df)}")

# Show missingness for key columns (we'll fix them next)
cols_to_check = ["event_type","state","begin_date_time","damage_property",
                 "magnitude","magnitude_type","begin_lat","begin_lon","month_name","begin_yearmonth"]
for c in cols_to_check:
    if c in df.columns:
        print(c, "→", df[c].isna().mean().round(3))


Duplicates removed: 0
event_type → 0.0
state → 0.0
begin_date_time → 0.0
damage_property → 0.221
magnitude → 0.458
magnitude_type → 0.612
begin_lat → 0.437
begin_lon → 0.437
month_name → 0.0
begin_yearmonth → 0.0


In [5]:
def parse_damage(x):
    if pd.isna(x): return 0.0
    s = str(x).strip().upper()
    if s == "" or s == "0.00K": return 0.0
    m = re.match(r"^([0-9.]+)\s*([KMB])$", s)
    if not m: 
        return np.nan
    val = float(m.group(1))
    mul = {"K":1e3, "M":1e6, "B":1e9}[m.group(2)]
    return val * mul

df["damage_property_num"] = df["damage_property"].apply(parse_damage)
df["damage_property_num"].describe()


count    3.390400e+04
mean     6.200466e+04
std      8.693729e+06
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.600000e+09
Name: damage_property_num, dtype: float64

In [6]:
# 6.1 try strict format first (NOAA spec: MM/DD/YYYY HH:MM:SS)
dt = pd.to_datetime(df.get("begin_date_time"), format="%m/%d/%Y %H:%M:%S", errors="coerce")

# 6.2 fallback parsing if needed
if dt.notna().sum() == 0:
    dt = pd.to_datetime(df.get("begin_date_time"), errors="coerce")

df["begin_date_time"] = dt
df["year"]  = df["begin_date_time"].dt.year
df["month"] = df["begin_date_time"].dt.month

# 6.3 fallback month from month_name
if df["month"].notna().sum() == 0 and "month_name" in df.columns:
    name_map = {'january':1,'february':2,'march':3,'april':4,'may':5,'june':6,
                'july':7,'august':8,'september':9,'october':10,'november':11,'december':12}
    df["month"] = df["month_name"].astype(str).str.lower().map(name_map)

# 6.4 fallback month from begin_yearmonth (YYYYMM)
if df["month"].notna().sum() == 0 and "begin_yearmonth" in df.columns:
    mm = pd.to_numeric(df["begin_yearmonth"].astype(str).str[-2:], errors="coerce")
    df["month"] = mm

# 6.5 final safety fill
if df["month"].notna().sum() == 0:
    df["month"] = 6  # default June if totally missing
else:
    df["month"] = df["month"].fillna(df["month"].median())

# 6.6 season
month_to_season = {12:"DJF",1:"DJF",2:"DJF", 3:"MAM",4:"MAM",5:"MAM",
                   6:"JJA",7:"JJA",8:"JJA", 9:"SON",10:"SON",11:"SON"}
df["season"] = df["month"].map(lambda m: month_to_season.get(int(m), "Unknown"))
df[["year","month","season"]].head()


C:\Users\sanjana\AppData\Local\Temp\ipykernel_6560\2277324538.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt = pd.to_datetime(df.get("begin_date_time"), errors="coerce")


,year,month,season
0,2025,3,MAM
1,2025,3,MAM
2,2025,1,DJF
3,2025,1,DJF
4,2025,1,DJF


In [8]:
def clean_str(s):
    if pd.isna(s): return np.nan
    return str(s).strip()

for c in ["event_type","state","magnitude_type"]:
    if c in df.columns:
        df[c] = df[c].apply(clean_str)

# States are uppercase in this dataset; ensure consistent casing
if "state" in df.columns:
    df["state"] = df["state"].str.upper()


In [9]:
if "begin_lat" in df.columns and "begin_lon" in df.columns:
    # Valid numeric
    df["begin_lat"] = pd.to_numeric(df["begin_lat"], errors="coerce")
    df["begin_lon"] = pd.to_numeric(df["begin_lon"], errors="coerce")

    # Drop rows with impossible coords
    valid = df["begin_lat"].between(-90, 90) & df["begin_lon"].between(-180, 180)
    before = len(df)
    df = df[valid].copy()
    print(f"Invalid lat/lon rows dropped: {before - len(df)}")


Invalid lat/lon rows dropped: 14806


In [10]:
feature_cols = ["event_type","state","month","season","magnitude","magnitude_type","begin_lat","begin_lon"]
target_col   = "damage_property_num"

missing_cols = [c for c in feature_cols+[target_col] if c not in df.columns]
assert not missing_cols, f"Missing needed columns: {missing_cols}"

df_model = df[feature_cols + [target_col]].copy()
df_model.head()


,event_type,state,month,season,magnitude,magnitude_type,begin_lat,begin_lon,damage_property_num
0,Thunderstorm Wind,GEORGIA,3,MAM,52.0,EG,33.4757,-85.238,1000.0
1,Tornado,MICHIGAN,3,MAM,NaN,NaN,41.7900,-86.100,100000.0
10,Flash Flood,TENNESSEE,4,MAM,NaN,NaN,36.0300,-89.330,0.0
11,Thunderstorm Wind,TENNESSEE,4,MAM,52.0,EG,36.1800,-88.160,0.0
12,Flash Flood,TENNESSEE,4,MAM,NaN,NaN,36.3000,-88.710,0.0


In [11]:
num_cols = ["month","magnitude","begin_lat","begin_lon"]
cat_cols = ["event_type","state","season","magnitude_type"]

for c in num_cols:
    df_model[c] = pd.to_numeric(df_model[c], errors="coerce")
    df_model[c] = df_model[c].fillna(df_model[c].median())

for c in cat_cols:
    df_model[c] = df_model[c].astype("string").fillna("Unknown")

# Target must be numeric and non-missing; drop rows where it's missing
before = len(df_model)
df_model = df_model[pd.to_numeric(df_model[target_col], errors="coerce").notna()].copy()
print(f"Dropped rows with missing target: {before - len(df_model)}")


Dropped rows with missing target: 0


In [12]:
#tame extreme outliers
cap = df_model[target_col].quantile(0.995)
df_model[target_col] = np.where(df_model[target_col] > cap, cap, df_model[target_col])
cap


np.float64(762874.9999999854)

In [13]:
print(df_model.shape)
df_model.isna().mean().sort_values().tail(10)
df_model.describe(include="all").T.head(20)


(19098, 9)


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
event_type,19098,14,Thunderstorm Wind,8736,NaN,NaN,NaN,NaN,NaN,NaN,NaN
state,19098,63,TEXAS,1983,NaN,NaN,NaN,NaN,NaN,NaN,NaN
month,19098.0,NaN,NaN,NaN,3.936852,1.039308,1.0,3.0,4.0,5.0,5.0
season,19098,2,MAM,17087,NaN,NaN,NaN,NaN,NaN,NaN,NaN
magnitude,19098.0,NaN,NaN,NaN,38.187374,23.378519,0.25,2.0,50.0,52.0,104.0
magnitude_type,19098,3,Unknown,9544,NaN,NaN,NaN,NaN,NaN,NaN,NaN
begin_lat,19098.0,NaN,NaN,NaN,36.565846,4.75826,-14.338,33.89,36.7106,39.58,61.7429
begin_lon,19098.0,NaN,NaN,NaN,-89.376379,10.168023,-170.8339,-95.02,-88.08,-82.75235,171.3065
damage_property_num,19098.0,NaN,NaN,NaN,10920.691172,67390.742112,0.0,0.0,0.0,1000.0,762875.0


In [14]:
out_parquet = OUT_DIR / "cleaned.parquet"
out_csv     = OUT_DIR / "cleaned.csv"

df_model.to_parquet(out_parquet, index=False)
df_model.to_csv(out_csv, index=False)
print("Saved:", out_parquet, "and", out_csv)


Saved: ..\data\processed\cleaned.parquet and ..\data\processed\cleaned.csv
